<a href="https://colab.research.google.com/github/jachiaram/GuitarML/blob/main/GuitarML.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Guitar Machine Learning Note Classifier Model V1**

## **Install Dependencies:**
In this section tensorflow is install if it isn't already present and other dependencies used in the notebook are imported here.

In [ ]:
!pip install tensorflow==2.8.0 tensorflow-gpu==2.8.0 tensorflow-io matplotlib

In [1]:
import os
from matplotlib import pyplot as plt
import tensorflow as tf
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelEncoder

## **Setting up Training and Testing Data for Model:**
In this section the csv file is read in and converted to a dataframe. Then that is split into X and y data, which is then split into training and testing data.

In [94]:
# df = pd.read_csv('https://raw.githubusercontent.com/jachiaram/GuitarML/main/Guitar_features.csv')
df = pd.read_csv('https://raw.githubusercontent.com/jachiaram/GuitarML/main/Guitar_features%20(1).csv')

In [96]:
y_column = 'note_names'
x_columns = ['chroma_stft', 'spectral_centroid', 'spectral_bandwidth', 'spectral_rolloff', 'spectral_contrast', 'mfcc', 'zero_crossing_rate', 'rms', 'tonnetz']

notes = df[y_column].values

label_encoder = LabelEncoder()

label_encoder.fit(notes)

df['Encoded_Note'] = label_encoder.transform(notes)

X = df[x_columns]
y = df['Encoded_Note']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

#y_train_numeric = label_encoder.fit_transform(y_train)
#y_test_numeric = label_encoder.fit_transform(y_test)

#y_train_one_hot = to_categorical(y_train_numeric, num_classes=37)

In [97]:
X_train.head()

,chroma_stft,spectral_centroid,spectral_bandwidth,spectral_rolloff,spectral_contrast,mfcc,zero_crossing_rate,rms,tonnetz
725,0.351038,585.169761,1527.327333,807.495117,21.517338,-10.584658,0.008418,0.060545,-0.171966
892,0.486339,448.798359,1413.824410,744.767960,17.894056,-2.661055,0.005658,0.100404,0.002941
1225,0.549486,954.273476,1593.178656,1897.262441,18.596631,-3.991036,0.008630,0.081227,0.139766
376,0.255876,913.123287,1941.985286,1313.057278,21.139120,-11.602310,0.014447,0.085229,-0.060925
1479,0.460173,668.133008,1779.425402,1173.559570,21.363525,-10.583671,0.005785,0.068602,0.098378


In [98]:
y_train.head()

725      1
892     35
1225    31
376      8
1479    36
Name: Encoded_Note, dtype: int64

## **Sequential Model Testing:**
In this section the model will be created, for the time being a sequential model is being used. The layers will need to be manipulated until a higher accuracy score is achieved (the goal is above 75%).


In [17]:
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense
from sklearn.metrics import accuracy_score

In [18]:
model = Sequential()
model.add(Dense(units=64, activation='relu', input_dim=len(X_train.columns)))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=256, activation='relu'))
model.add(Dense(units=128, activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=37, activation='softmax'))

In [19]:
from keras.optimizers import Adam
optimizer = Adam(learning_rate=0.001)
model.compile(optimizer=optimizer, loss='categorical_crossentropy', metrics=['accuracy'])

In [21]:
model.fit(X_train, y_train_one_hot, epochs=50, batch_size=10)

Epoch 1/50
182/182 [==============================] - 1s 6ms/step - loss: 3.6345 - accuracy: 0.0281
Epoch 2/50
182/182 [==============================] - 1s 6ms/step - loss: 3.6826 - accuracy: 0.0264
Epoch 3/50
182/182 [==============================] - 1s 5ms/step - loss: 3.6186 - accuracy: 0.0270
Epoch 4/50
182/182 [==============================] - 1s 5ms/step - loss: 3.6109 - accuracy: 0.0264
Epoch 5/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6107 - accuracy: 0.0275
Epoch 6/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6106 - accuracy: 0.0303
Epoch 7/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6104 - accuracy: 0.0303
Epoch 8/50
182/182 [==============================] - 1s 4ms/step - loss: 3.6103 - accuracy: 0.0303
Epoch 9/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6102 - accuracy: 0.0303
Epoch 10/50
182/182 [==============================] - 1s 3ms/step - loss: 3.6102 - accuracy: 0.0303

In [22]:
from tensorflow.keras.utils import to_categorical
# Convert categorical test labels to one-hot encoding
y_test_one_hot = to_categorical(label_encoder.transform(y_test), num_classes=37)

# Evaluate the model on the test set
loss, accuracy = model.evaluate(X_test, y_test_one_hot)

print(f'Test Loss: {loss:.4f}')
print(f'Test Accuracy: {accuracy:.4f}')

15/15 [==============================] - 0s 2ms/step - loss: 3.6283 - accuracy: 0.0132
Test Loss: 3.6283
Test Accuracy: 0.0132


## **Random Forest Classifier Testing:**
In this section I was experimenting with a Random Forest Classifier to see if I could get any better results with this model versus the other one.

In [99]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split

# Assuming X_train and y_train are your training data and labels
# Assuming X_test and y_test are your test data and labels

# Convert string labels to numeric
# label_encoder = LabelEncoder()
# y_train_encoded = label_encoder.fit_transform(y_train)
# y_test_encoded = label_encoder.fit_transform(y_test)
# y_train_encoded

In [143]:
# Initialize the Random Forest classifier
rf_classifier = RandomForestClassifier(max_depth=20, min_samples_split=5, min_samples_leaf=18, max_features='log2', n_estimators=410, random_state=43)

In [144]:
# Train the Random Forest classifier
# rf_classifier.fit(X_train, y_train_encoded)
rf_classifier.fit(X_train, y_train)

RandomForestClassifier(max_depth=20, max_features='log2', min_samples_leaf=18,
                       min_samples_split=5, n_estimators=410, random_state=43)

In [145]:
# Make predictions on the test set
y_pred = rf_classifier.predict(X_test)

In [146]:
# Convert predictions back to original labels
#y_pred_original = label_encoder.inverse_transform(y_pred)

# Evaluate accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy}")

Accuracy: 0.7180616740088106
